In [1]:
from pathlib import Path

import util
from util import workflow

browser = True
file = util.notebook_file() if util.is_notebook() else __file__
tag = util.file_tag(file)
root_path = Path("..")
data_path = util.data_path(root_path)

In [2]:
# Build
from automol.graph import enum

import automech
from automech.species import Species

mech0 = automech.io.read(data_path / "full_raw.json")
mech = automech.from_smiles(
    spc_smis=["C12C(O2)[CH]CC1", "C12C(O2)C[CH]C1"],
    src_mech=mech0,
)
#  - O2 additions
mech = automech.enumerate_reactions(
    mech,
    enum.ReactionSmarts.o2_addition,
    rcts_=[None, "[O][O]"],
    spc_col_=Species.smiles,
    src_mech=mech0,
)
#  - HO2 elimination
mech = automech.enumerate_reactions(
    mech,
    enum.ReactionSmarts.ho2_elimination,
    spc_col_=Species.smiles,
    src_mech=mech0,
)
#  - RO2 -> QOOH
mech = automech.enumerate_reactions(
    mech,
    enum.ReactionSmarts.qooh_formation,
    spc_col_=Species.smiles,
    src_mech=mech0,
)
#  - QOOH beta scission to produce HO2
mech = automech.enumerate_reactions(
    mech,
    enum.ReactionSmarts.qooh_beta_scission,
    spc_col_=Species.smiles,
    src_mech=mech0,
)
#  - QOOH ring-forming scission to produce OH
mech = automech.enumerate_reactions(
    mech,
    enum.ReactionSmarts.qooh_ring_forming_scission,
    spc_col_=Species.smiles,
    src_mech=mech0,
)

  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
# Write
workflow.write(mech=mech, tag=tag, root_path=root_path, browser=browser)


Finalizing build for...
reactions=shape: (20, 5)
┌─────────────┬─────────────────────────┬───────────┬────────────┬─────────────────────────────────┐
│ reactants   ┆ products                ┆ formula   ┆ reversible ┆ rate_constant                   │
│ ---         ┆ ---                     ┆ ---       ┆ ---        ┆ ---                             │
│ list[str]   ┆ list[str]               ┆ struct[3] ┆ bool       ┆ struct[17]                      │
╞═════════════╪═════════════════════════╪═══════════╪════════════╪═════════════════════════════════╡
│ ["S(1704)"] ┆ ["S(1690)"]             ┆ {5,7,3}   ┆ true       ┆ {1,{null,null,null,null,null,n… │
│ ["S(1690)"] ┆ ["C5O3ddrkjl"]          ┆ {5,7,3}   ┆ null       ┆ null                            │
│ ["S(1690)"] ┆ ["S(1703)"]             ┆ {5,7,3}   ┆ true       ┆ {1,{null,null,null,null,null,n… │
│ ["S(1690)"] ┆ ["S(1702)"]             ┆ {5,7,3}   ┆ true       ┆ {1,{null,null,null,null,null,n… │
│ ["S(1700)"] ┆ ["S(1691)"]             ┆

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/51 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

InvalidOperationError: `old` input for `replace` must not contain duplicates

In [ ]:
# # Read
# workflow.read(tag=tag, root_path=root_path)

In [ ]:
# # Simulate
# workflow.simulate(full_tag=f"full_{tag}_calc", root_path=root_path)
# workflow.simulate(full_tag=f"full_{tag}_control", root_path=root_path)

In [ ]:
# # Plot
# charts = workflow.plot(
#     full_tag=f"full_{tag}_calc",
#     x_col="O2_molecules",
#     y_col_=["C5H8(522)", "C5H8O(825)rs"],
#     root_path=root_path,
#     line_source_=["hill", "lokachari"],
#     point_source="experiment",
# )
# for chart in charts:
#     chart.show()